In [1]:
# imports
import math
import numpy as np
import os
import random


def simulate_haplotypes(read_length, std_read_length, 
                        coverage, ref_length=int(2.5e2),
                        false_variance=0.0,
                        switch_error_rate=0.0,
                        miscall_error_rate=0.0,
                        missing_error_rate=0.0):
    """
    param read_length: Avg read length.
    param std_read_length How the read lengths is distributed around the read_length, which is avg read length. 
    param coverage: Number of avegage reads per genome for the reference over snps
    param reference_length: THe length of the reference haptype 
    param false_variance:
    param switch_error_rate: 
    param missing_error_rate:
    param miscall_error_rate:
    """
    read_length_bak = read_length

    read_length = int(read_length)
    mate_length = read_length
    num_reads_for_read_coverage = int(ref_length * coverage / read_length)
    
    # Create a reference Haplotype:
    p = 0.5
    reference_hap1 = np.random.choice(a=[False, True], size=(ref_length), p=[p, 1-p])
    reference_hap2 = np.logical_not(reference_hap1)
    haps = [reference_hap1, reference_hap2]
    
    # False Variance 
    # Lets assume there is false variance for the sequence with some probablity.


    # The sequence haptype we get has some false variance, i.e. some of the 
    # snps are homologous. 


    # sequencing error is a common error, we should simulate the sequencing 
    # error too. 
  
    
    # Pick the length of the read length from a normal distribution centered at
    # read_lenth and variance as std_read_length
    read_lengths = np.array(
        np.random.normal(loc = read_length, scale = std_read_length, 
                         size = num_reads_for_read_coverage), 
                         dtype="int32")
    
    # Uniformy choose start points from the reference haplotype, h1 or h2. 
    reads_start_idx = np.random.randint(low = 0, high=ref_length, size=num_reads_for_read_coverage)
    
    reads_st_en = []
    #reads = np.array([reads_start_idx, reads_start_idx + read_length])
    for st_idx, read_len in zip(reads_start_idx, read_lengths):
      if st_idx + read_len > ref_length - 1:
        en = ref_length - 1
      else:
        en = st_idx + read_len
      reads_st_en.append( (st_idx, en ) )
    #reads_st_en = np.array(reads_st_en)
    #import pdb;pdb.set_trace()
    # We have the st and en of the reads, choose either h1 or h2 from H and 
    # sample it. Sample from S1 if True else sample from S2.
    h1_or_h2 = np.array(
        np.less(np.random.uniform(0, 1, num_reads_for_read_coverage), 0.5), 
        dtype="int")
     
    hap_samples = [haps[v][st:en] for v, (st, en) in zip(h1_or_h2, reads_st_en)]
    #pdb.set_trace()
    # Simulate switch errors:

    sw_hap_samples = []
    fragfilecontent = []
    # Get the switch error for each of the sample.
    # Get the missing error for each of the sample 
    qual = '~' if miscall_error_rate < 1e-9 else str(int(-10*math.log10(miscall_error_rate)))
    
    
    for sample, x in zip(hap_samples, reads_st_en ):
      #import pdb;pdb.set_trace()  
      
      assert len(sample) == abs(x[1] - x[0])  
      switch_error = np.less( np.random.uniform(0, 1, size=len(sample)) , switch_error_rate)
      miscall_error = np.less(np.random.uniform(0, 1, size=len(sample)) , miscall_error_rate)      
      missing_error = np.less(np.random.uniform(0, 1, size=len(sample)) , missing_error_rate)
    
      # Simulate switch errors
      switch_idxs = list(np.nonzero(switch_error))
      is_switched = False
      new_sample = []
      for sa, sw in zip(sample, switch_error):
        if sw:
          is_switched = not is_switched
          if is_switched:
            new_sample.append(not sa)
          else:
            new_sample.append(sa) 
        else:
            new_sample.append(sa) 
            
      updated_sample = new_sample
      assert len(updated_sample) == abs(x[1] - x[0])

      # Simulate miscall errors      
      new_sample = []
      for sa, miscall in zip(updated_sample, miscall_error):
        if miscall:
          new_sample.append(not sa)
        else:
          new_sample.append(sa)
      
      updated_sample = new_sample 
      assert len(updated_sample) == abs(x[1] - x[0])

      # Simulate missing errors  
      new_sample = []

      for sa, missing in zip(updated_sample, missing_error):
        if missing:
          new_sample.append(-1)
        else:
          new_sample.append(sa)
      assert len(new_sample) == abs(x[1] - x[0])
      # INdex of the variant, reference hap value, misscall rate
        
      fragfilecontent.append((x, new_sample, qual))      
      sw_hap_samples.append(new_sample)
        
    #import pdb;pdb.set_trace()
    # Update the hap samples with the new samples
    hap_samples = [list(np.array(s, dtype="int32")) for s in sw_hap_samples]
    

    assert len(hap_samples) == num_reads_for_read_coverage
    return hap_samples, reads_st_en, fragfilecontent, haps

In [2]:
ref_length = int(30)
hap_samples, st_en, frag_file_contents, ref_H = simulate_haplotypes(read_length=5, std_read_length=2, 
                                                                coverage=3, ref_length=ref_length,
                                                                false_variance=0.1,
                                                                switch_error_rate=0.1,
                                                                miscall_error_rate=0.1,
                                                                missing_error_rate=0.02)

In [3]:
miscall_error_rate = 0.02
q = '~' if miscall_error_rate < 5.011872336272714e-10 else chr(int(33-10*math.log10(miscall_error_rate)))
for x, sa in zip(st_en, hap_samples):
    print(x, sa, q)

(18, 21) [1, 0, 1] 1
(5, 8) [1, 1, 1] 1
(11, 17) [1, 1, 1, 1, 1, 0] 1
(7, 11) [0, 0, 1, 0] 1
(4, 9) [0, 0, 1, 0, 0] 1
(15, 18) [1, 0, 0] 1
(13, 17) [0, 1, 1, 0] 1
(1, 5) [0, 1, 0, 1] 1
(10, 13) [1, 1, 0] 1
(23, 28) [0, 0, 1, 1, 0] 1
(14, 20) [1, 1, 0, 1, 1, 1] 1
(22, 27) [0, 0, 0, 1, 1] 1
(5, 11) [1, 1, 0, 0, 0, 1] 1
(9, 12) [1, 0, 0] 1
(20, 24) [1, 0, 1, 0] 1
(0, 5) [0, 0, 0, 0, 0] 1
(12, 15) [1, 1, 1] 1
(29, 29) [] 1


In [4]:
def generate_matrix_for_visualization(hap_samples, reference_length, st_en):
    matrix = [["-" for _ in range(reference_length)] for _ in range(len(hap_samples))]
    for idx, ( (s,e),  sa ) in enumerate(zip(st_en, hap_samples)):
        for i, v in zip(range(s, e), sa):
            if v != -1:
                matrix[idx][i] = v
                
    for m in matrix:
        print(" ".join(str(v) for v in m) )
        
        
        
generate_matrix_for_visualization(hap_samples, 30, st_en) 

- - - - - - - - - - - - - - - - - - 1 0 1 - - - - - - - - -
- - - - - 1 1 1 - - - - - - - - - - - - - - - - - - - - - -
- - - - - - - - - - - 1 1 1 1 1 0 - - - - - - - - - - - - -
- - - - - - - 0 0 1 0 - - - - - - - - - - - - - - - - - - -
- - - - 0 0 1 0 0 - - - - - - - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - - 1 0 0 - - - - - - - - - - - -
- - - - - - - - - - - - - 0 1 1 0 - - - - - - - - - - - - -
- 0 1 0 1 - - - - - - - - - - - - - - - - - - - - - - - - -
- - - - - - - - - - 1 1 0 - - - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - - - - - - - - - - 0 0 1 1 0 - -
- - - - - - - - - - - - - - 1 1 0 1 1 1 - - - - - - - - - -
- - - - - - - - - - - - - - - - - - - - - - 0 0 0 1 1 - - -
- - - - - 1 1 0 0 0 1 - - - - - - - - - - - - - - - - - - -
- - - - - - - - - 1 0 0 - - - - - - - - - - - - - - - - - -
- - - - - - - - - - - - - - - - - - - - 1 0 1 0 - - - - - -
0 0 0 0 0 - - - - - - - - - - - - - - - - - - - - - - - - -
- - - - - - - - - - - - 1 1 1 - - - - - 

In [5]:
#print(frag_file_contents)

In [6]:
def print_fragment(frag_data):
    # build line content
    # INdex of the variant, reference hap value, misscall rate
    frags = []
    for (st,en), alleles, qual in frag_data:
        fs1 = [(idx, str(int(val)), qual) for idx, val in zip(range(st, en), alleles)]
        frags.append(fs1)
           
    for idx, fs in enumerate(frags):
        name = "FRAG{}".format(idx+1)
        fragstr = ''
        num_pairs = 0
        prev_snp_ix = -2
        qual = ' '
        for snp_ix, allele, q_char in fs:

            diff = snp_ix - prev_snp_ix

            if diff == 1:
                fragstr += allele
            else:
                num_pairs += 1
                fragstr += ' {} {}'.format(snp_ix+1, allele)

            prev_snp_ix = snp_ix
            qual += q_char

        fragstr += qual

        prefix = '{} {}'.format(num_pairs,name)
        fragstr = prefix + fragstr

        # print line to file
        print(fragstr)


    # with open("sample.fragments", "r") as fd:
    #   lines = fd.readlines()
    # print("".join(lines))  
print_fragment(frag_file_contents)

1 FRAG1 19 101 101010
1 FRAG2 6 111 101010
1 FRAG3 12 111110 101010101010
1 FRAG4 8 0010 10101010
1 FRAG5 5 00100 1010101010
1 FRAG6 16 100 101010
1 FRAG7 14 0110 10101010
1 FRAG8 2 0101 10101010
1 FRAG9 11 110 101010
1 FRAG10 24 00110 1010101010
1 FRAG11 15 110111 101010101010
1 FRAG12 23 00011 1010101010
1 FRAG13 6 110001 101010101010
1 FRAG14 10 100 101010
1 FRAG15 21 1010 10101010
1 FRAG16 1 00000 1010101010
1 FRAG17 13 111 101010
0 FRAG18 


In [7]:
def print_vcf_file_format(st_en, H):
    ref_name = "ref_name"
    ref_length = len(H[0])
    header ='''##fileformat=VCFv4.1
##contig=<ID={},length={}>
##INFO=<ID=mt,Number=1,Type=String,Description="Variant Type: SUBSTITUTE/INSERT/DELETE">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=FP,Number=1,Type=Integer,Description="Read Depth">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SIM_INDIVIDUAL'''.format(ref_name,ref_length)
    
    print(header)
    bases = ['A','T','G','C']
    for snp in range(ref_length):
        ref_ix = random.randrange(0,4)
        alt_ix = ref_ix
        while(alt_ix == ref_ix):
            alt_ix = random.randrange(0,4)

        genotype_field = '{}|{}'.format(int(H[0][snp]),int(H[1][snp]))
        ID = '.'
        ref_snp = bases[ref_ix]
        alt_snp = bases[alt_ix]
        qual = 100
        fltr = 'PASS'
        info = 'mt=SUBSTITUTE'
        print('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\tGT\t{}'.format(ref_name, snp, ID, ref_snp, alt_snp, qual, fltr, info, genotype_field))
    # with open("sample.vcf", "r") as fd:
    #   lines = fd.readlines()
    # print(len(lines))  
    # print("".join(lines))   
print_vcf_file_format(st_en,  ref_H)           

##fileformat=VCFv4.1
##contig=<ID=ref_name,length=30>
##INFO=<ID=mt,Number=1,Type=String,Description="Variant Type: SUBSTITUTE/INSERT/DELETE">
##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">
##FORMAT=<ID=FP,Number=1,Type=Integer,Description="Read Depth">
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	SIM_INDIVIDUAL
ref_name	0	.	G	A	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	1	.	T	C	100	PASS	mt=SUBSTITUTE	GT	1|0
ref_name	2	.	A	C	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	3	.	C	G	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	4	.	G	A	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	5	.	C	T	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	6	.	C	A	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	7	.	T	C	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	8	.	G	C	100	PASS	mt=SUBSTITUTE	GT	1|0
ref_name	9	.	A	T	100	PASS	mt=SUBSTITUTE	GT	1|0
ref_name	10	.	A	G	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	11	.	A	C	100	PASS	mt=SUBSTITUTE	GT	0|1
ref_name	12	.	G	C	100	PASS	mt=SUBSTITUTE	GT	1|0
ref_name	13	.	T	G	100	PASS	mt=SUBSTITUTE	GT	1|0
ref_name	

Notes: We want to incorporate sequencing errors in the fragments f1 and f2 and get the confidence whether f1 and f2 belong from the same haplotype. Use some probablity to guess whether f1 and f2 belong to the same haplotype. 

Now once we have a probablity that f1 and f2 belong to the same haplotype, we want to check whether a variant is a false variant.

What is a false variant ? 
False variant is a location where the alleles are homozygous but the sequencing 
assumes / wrongly interprets the site as heterozygous. 
Hitherto, in the fragments (for these false variants) there would be a substantial amount of fragments where this site has the same allele say 0 for 
both fragments coming from both H1 and H2 (as it is not really a variant hence same allele is observed irrespective sample from H1 or H2 (0, 0) in the given position).

However for sequencing errors the errors are randomly distributed among different variants for different fragments.

Problem statement, we want to identify these false variant locations for the 
sequenced data, as these become difficult to handle while performing Hap assembly.

When we have a good coverage for a particular false variant, 
if we have fragment f1 which belongs to H1 and a fragment f2 which belongs to H2, the value for the variants are complement of each other except for the false variant. 

Now we dont know whether the fragment belongs to either H1 or H2, 